### Authenticate
This is going to have to change for prod since we won't have access to user passwords

In [ ]:
import { authenticate } from './authentication.ts'
import { GATEWAY } from './helpers/routes.ts'

const token = await authenticate(GATEWAY, 'k.mweene', 'test') //TODO we're going to need a better way to do this in prod. They can just create us a system client
const adminToken = await authenticate(GATEWAY, 'j.campbell', 'test')
token


### Register API client
For prod, best option will probably be to get a client created for us by the local team and just get the clientId and Secret and store them as github secrets and pass them in as deno args.

In [ ]:
import { registerSystem } from './helpers/gqlHandlers.ts'

const systemRegistration = await registerSystem(adminToken)
const clientId = systemRegistration.data.registerSystem.system.clientId
const clientSecret = systemRegistration.data.registerSystem.clientSecret


In [ ]:
async function getTokenForSystemClient(
  clientId: string,
  clientSecret: string
): Promise<string> {
  const authenticateResponse = await fetch(
    `${GATEWAY}/auth/token?client_id=${clientId}&client_secret=${clientSecret}&grant_type=client_credentials`,
    {
      method: 'POST',
      headers: {
        'Content-Type': 'application/json',
        'x-correlation-id': clientId + '-' + Date.now(),
      },
    }
  )
  const res = await authenticateResponse.json()

  return res.token || res.access_token
}

const sysToken = await getTokenForSystemClient(clientId, clientSecret)


### Fetch registrations

In [ ]:
import { fetchAllBirthRegistrations } from './helpers/gqlHandlers.ts'

const birthRegistrations = await fetchAllBirthRegistrations(token)
const birthIds = birthRegistrations.data.searchEvents.results.map((x) => x.id)

birthIds


In [ ]:
import { fetchAllDeathRegistrations } from './helpers/gqlHandlers.ts'

const deathRegistrations = await fetchAllDeathRegistrations(token)
const deathIds = deathRegistrations.data.searchEvents.results.map((x) => x.id)
deathIds


### Fetch birth and death events

In [ ]:
import { fetchEvents } from './helpers/formsHandlers.ts'
import { extractFieldType } from './helpers/utils.ts'

const events = await fetchEvents(token)

const ignoredFields = ['DIVIDER', 'PARAGRAPH']

const birthEvent = events.find((x) => x.id === 'v2.birth')
const deathEvent = events.find((x) => x.id === 'v2.death')

const birthEventFields = new Set(
  extractFieldType(birthEvent, 'fields')
    .filter((x) => !ignoredFields.includes(x.type))
    .map((f) => f.id)
    .filter((x) => x)
)

const deathEventFields = new Set(
  extractFieldType(deathEvent, 'fields')
    .filter((x) => !ignoredFields.includes(x.type))
    .map((f) => f.id)
    .filter((x) => x)
)


In [ ]:
const declareEvent = async (document, sysToken) => {
  const response = await fetch(`${GATEWAY}/events/event.import`, {
    method: 'POST',
    headers: {
      'Content-Type': 'application/json',
      Authorization: `Bearer ${sysToken}`,
    },
    body: JSON.stringify({
      json: document,
      meta: {
        values: {
          declaration: ['undefined'],
        },
      },
    }),
  })

  if (!response.ok) {
    console.log((await response.json()).error)

    throw new Error(`Event creation failed: ${response.statusText}`)
  }
  return response.json()
}


### Get all potential resolvers
Use only resolvers for used event fields to avoid nulls

In [ ]:
import defaultResolvers from './helpers/defaultResolvers.ts'
import { countryResolver } from './helpers/countryResolvers.ts'

const allResolvers = { ...defaultResolvers, ...countryResolver }

const birthResolver = Object.fromEntries(
  Object.entries(allResolvers).filter(([key, _value]) =>
    [...birthEventFields].includes(key)
  )
)

const deathResolver = Object.fromEntries(
  Object.entries(allResolvers).filter(([key, _value]) =>
    [...deathEventFields].includes(key)
  )
)


### Migrate births

In [ ]:
import { fetchBirthRegistration } from './helpers/gqlHandlers.ts'
import { transform } from './transform.ts'

const saved = []

const migrateBirth = async (entryId) => {
  // Get entry
  const birthRegistrationData = await fetchBirthRegistration(entryId, token)
  //transform
  const document = transform(
    birthRegistrationData.data.fetchBirthRegistration,
    birthResolver
  )
  //validate
  //post
  const result = await declareEvent(document, sysToken)
  //throw if it fails?

  return result.result.data.json.id
}

for (const id of birthIds) {
  const savedId = await migrateBirth(id)
  saved.push(savedId)
}


### Migrate Deaths

In [ ]:
import { fetchDeathRegistration } from './helpers/gqlHandlers.ts'

const migrateDeath = async (entryId) => {
  // Get entry
  const deathRegistrationData = await fetchDeathRegistration(entryId, token)
  //transform
  const document = transform(
    deathRegistrationData.data.fetchDeathRegistration,
    deathResolver
  )
  //validate
  //post
  const result = await declareEvent(document, sysToken)
  //throw if it fails?

  return result.result.data.json.id
}

for (const id of deathIds) {
  const savedId = await migrateDeath(id)
  saved.push(savedId)
}


### Output results
Could make these better to make testing easier

In [ ]:
import { REGISTER_APP } from './helpers/routes.ts'

console.log('🍞 Declarations succesfully migrated:')
console.log()
saved.forEach((s) => {
  console.log(
    `You can view the original record [here](${REGISTER_APP}/${s}/viewRecord?V2_EVENTS=false)`
  )
  console.log(
    `and the new migrated record [here](${REGISTER_APP}/events/view/${s}?V2_EVENTS=true)`
  )
})
